In [2]:
import re
import pandas as pd
import numpy as np

## Task 1: Mở file

In [306]:
# Vì số câu trả lời có thể lớn hơn 25 nên không mở bằng read_csv như bình thường.
# mở file bằng read_fwf và đặt dữ liệu vào cột tên raw_data
while True:
    try:
        file_name = input("Enter a class file to grade (i.e. class1 for class1.txt): ")
        df = pd.read_fwf(file_name, header=None)
        print(f"Successfully opened {file_name}")
        df.columns = ['Raw_data']
        print(df.head())
        break
    except:
        print("File cannot be found")

Successfully opened class8.txt
                                            Raw_data
0  N00002142,B,A,D,A,C,B,D,A,C,C,D,B,A,B,,C,B,D,A...
1  N00002143,,A,A,D,C,B,,B,C,C,,B,A,D,,B,B,B,A,C,...
2  N00002144,B,A,C,D,C,B,D,A,C,C,D,B,A,B,A,,B,D,A...
3  N00002145,B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,,D,D,A...
4  N00002146,B,A,A,B,C,B,D,A,C,,C,B,,B,C,C,B,D,A,...


## Task 2: Đưa ra các giá trị sai định dạng

In [307]:
# Tạo ra 1 cột chứa giá trị là 1 list được tạo bởi raw_data
df['Split_data'] = df.Raw_data.str.strip().str.split(',')

# Đưa ra các dòng sai định dạng khác 26 giá trị
invalid_1 = df.loc[df['Split_data'].apply(lambda x: len(x)) != 26]['Raw_data']
num_inva1 = len(invalid_1)

if num_inva1 != 0:
    print(f"There are {num_inva1} Invalid lines do not contain exactly 26 values:")
    for _ in invalid_1:
        print(_)
else:
    print("There is no invalid line does not contain exactly 26 values")

There is no invalid line does not contain exactly 26 values


In [308]:
# Đưa ra các dòng sai định dạng là tên học sinh bị sai
invalid_2 = df.loc[~df['Raw_data'].str.contains('N[0-9]{8}')]['Raw_data']
num_inva2 = len(invalid_2)

if num_inva2 != 0:
    print(f"There are {num_inva2} Invalid lines have wrong name")
    for _ in invalid_2:
        print(_)
else:
    print("There is no invalid line has wrong name")

There is no invalid line has wrong name


### Đưa ra kết luận về dữ liệu

In [309]:
print(f"Total valid lines of data {df.shape[0] - num_inva1 - num_inva2}")
print(f"Total invalid lines of data {num_inva1 + num_inva2}")

Total valid lines of data 49
Total invalid lines of data 0


## Task 3: Chấm điểm cho các bài thi

In [310]:
# Loại bỏ các dòng invalid
invalid_index = invalid_1.index.append(invalid_2.index)
df_valid = df.drop(index=invalid_index)


### Tạo 1 DataFrame mới phân chia kết quả ra từng cột

In [311]:
# Tạo dataframe
answer = df_valid['Split_data']
df_answer = pd.concat([answer.apply(pd.Series)], axis=1)

df_answer.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,N00002142,B,A,D,A,C,B,D,A,C,...,C,B,D,A,C,A,A,B,D,
1,N00002143,,A,A,D,C,B,,B,C,...,B,B,B,A,C,A,A,B,D,D
2,N00002144,B,A,C,D,C,B,D,A,C,...,,B,D,A,C,A,A,B,,D
3,N00002145,B,A,D,D,C,B,D,A,C,...,,D,D,A,C,A,A,B,D,B
4,N00002146,B,A,A,B,C,B,D,A,C,...,C,B,D,A,C,A,A,,D,D


### Tính điểm cho từng học sinh

In [312]:
# Answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
lst_answer = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D".split(',')

# tạo 1 hàm để tính điểm
def score_check(a,b):
    score = 0
    if a == '':
        score = 0
    elif a == b:
        score = 4
    else:
        score = -1
    return score

df_score = pd.DataFrame(df_answer[0])

for i in range(1,26):
    df_score[i] = df_answer[i].apply(lambda x: score_check(x,lst_answer[i-1]))

df_score['Total'] = df_score[[i for i in range(1,26)]].sum(axis=1)
df_score.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,Total
0,N00002142,4,4,4,-1,4,4,4,4,4,...,4,4,4,4,4,4,4,4,0,87
1,N00002143,0,4,-1,4,4,4,0,-1,4,...,4,-1,4,4,4,4,4,4,4,59
2,N00002144,4,4,-1,4,4,4,4,4,4,...,4,4,4,4,4,4,4,0,4,87
3,N00002145,4,4,4,4,4,4,4,4,4,...,-1,4,4,4,4,4,4,4,-1,86
4,N00002146,4,4,-1,-1,4,4,4,4,4,...,4,4,4,4,4,4,0,4,4,68


### Đưa ra các thông số

In [313]:
# Số lượng học sinh đạt điểm cao
number_of_high_score_student = (df_score.Total > 80).sum()
print(f"Total student of high scores : {number_of_high_score_student}")

# Điểm trung bình
avg_score = df_score.Total.mean()
print("Mean (average) score: {:.2f}".format(avg_score))

# Điểm cao nhất
highest_score = df_score.Total.max()
print(f"Highest score: {highest_score}")

# Điểm thấp nhất
lowest_score = df_score.Total.min()
print(f"Lowest score: {lowest_score}")

# Miền giá trị của điểm
Range_of_score = highest_score - lowest_score
print(f"Range of score: {Range_of_score}")

# Giá trị trung vị
Median_score = df_score.Total.median()
print(f"Median score: {Median_score}")

# Thống kê xem câu hỏi nào bị bỏ qua nhiều nhất
se_skip_que = df_score.apply(lambda x: x.value_counts().get(0))
skip_max_count = se_skip_que.max()
skip_que = se_skip_que[se_skip_que == skip_max_count]

skip_str = ""
for _ in skip_que.index:
    skip_str += "{} - {} - {:.2f}, ".format(_, skip_max_count, skip_max_count/(df_valid.shape[0]))
print("Question that most people skip:", skip_str.strip(', '))

#Thống kê xem câu hỏi nào bị trả lời sai nhiều nhất
se_skip_que = df_score.apply(lambda x: x.value_counts().get(-1))
skip_max_count = se_skip_que.max()
skip_que = se_skip_que[se_skip_que == skip_max_count]

skip_str = ""
for _ in skip_que.index:
    skip_str += "{} - {} - {:.2f}, ".format(_, skip_max_count, skip_max_count/(df_valid.shape[0]))
print("Question that most people answer incorrectly:", skip_str.strip(', '))

Total student of high scores : 17
Mean (average) score: 76.67
Highest score: 92
Lowest score: 59
Range of score: 33
Median score: 77.0
Question that most people skip: 4 - 9.0 - 0.18
Question that most people answer incorrectly: 4 - 9.0 - 0.18, 15 - 9.0 - 0.18


## Task 4: Xuất dataframe ra file .txt

In [314]:
write_file_name = file_name.strip('.txt') + "_grades.txt"
df_score[[0,'Total']].to_csv(write_file_name, index=False, header=False)